In [1]:
import os
import folium
import numpy as np
import pandas as pd
import math
from collections import Counter
import matplotlib
import matplotlib.pyplot as plt
from scipy.ndimage.interpolation import shift
import scipy.stats as stats
from scipy.stats import norm
import matplotlib.mlab as mlab
import pylab as pl
from folium.plugins import HeatMap

%matplotlib inline
%config InlineBackend.figure_format = 'svg'

In [2]:
battery_gps_df = pd.read_csv("battery_gps.csv")
battery_gps_df.record_time = battery_gps_df.record_time.astype('datetime64[s]')

In [3]:
battery_gps_df.dtypes

record_time    datetime64[ns]
user_id                 int64
level                   int64
plugged                 int64
accu                  float64
lat                   float64
lon                   float64
grid_x                  int64
grid_y                  int64
center_lon            float64
center_lat            float64
is_home                 int64
dtype: object

In [4]:
def home_charge_percentage(df):
    charge_df = df[df['plugged'] != 0]
    home_charge_df = charge_df[charge_df['is_home'] == 1]
    
    charge_counts = charge_df.groupby(['user_id']).size().reset_index(name='total_count')
    home_charge_counts = home_charge_df.groupby(['user_id']).size().reset_index(name='count')
    home_charge_counts.set_index('user_id', inplace=True)
    
    merged_counts = pd.merge(home_charge_counts, charge_counts, on='user_id', how='outer')
    merged_counts.loc[:, 'home_charge_perc'] = merged_counts['count'] / merged_counts['total_count']
    
    merged_counts = merged_counts.drop(columns = ['count', 'total_count'])
    
    return merged_counts

In [5]:
def plug_in_time_percentage(df):
    plug_counts = df.groupby(['user_id', 'plugged']).size().reset_index(name='count')
    total_counts = df.groupby(['user_id']).size().reset_index(name='total_count')
    
    plug_counts.set_index('user_id', inplace=True)
    plug_counts = plug_counts.pivot(index=plug_counts.index, columns='plugged')['count']
    print(plug_counts.head())
    plug_counts.columns = ['uncharged', 'charging_mode1', 'charging_mode2']
    plug_counts = plug_counts.fillna(0)
    
    merged_counts = pd.merge(plug_counts, total_counts, on='user_id', how='outer')
    merged_counts.loc[:, 'uncharge_perc'] = merged_counts['uncharged'] / merged_counts['total_count']
    merged_counts.loc[:, 'charging_mode1_perc'] = merged_counts['charging_mode1'] / merged_counts['total_count']
    merged_counts.loc[:, 'charging_mode2_perc'] = merged_counts['charging_mode2'] / merged_counts['total_count']
    merged_counts = merged_counts.drop(columns = ['uncharged', 'charging_mode1', 'charging_mode2', 'total_count'])
    
    # return a dataframe that contains user_id and aggregated features
    return merged_counts

In [6]:
def mean_length_charge_per_user(df):
    df.loc[df.plugged != 0, 'plugged'] = 1 
    df.sort_values(['user_id', 'record_time'])
    df = df.drop(columns=['level', 'accu','lat', 'lon'])

    df.loc[:, 'plugged_change'] = shift(df.plugged, -1, cval=0) - df['plugged']
    
    df.loc[:, 'temp'] = df.plugged_change.cumsum()
    df['temp'] = 1 - df['temp']
    df.loc[:, 'cumsum_of_temp'] = df.temp.cumsum()
    
    length_charge = df.groupby(['user_id', 'cumsum_of_temp']).size().reset_index(name='charge_length')
    length_charge = length_charge.drop(length_charge[length_charge.charge_length <= 1].index)
    length_charge = length_charge.drop(columns = ['cumsum_of_temp'])
    
    mean_length_charge = length_charge.groupby(['user_id']).mean().reset_index()
    
    # return mean charge length in battery record unit. We can convert it into minute by multiplying 5 minutes
    return mean_length_charge

In [7]:
def num_charges_per_user(df):
    df.loc[df.plugged != 0, 'plugged'] = 1 
    df.sort_values(['user_id', 'record_time'])
    df = df.drop(columns=['level', 'accu','lat', 'lon'])

    
    df.loc[:, 'plugged_change'] = shift(df.plugged, -1, cval=0) - df['plugged']
    df.loc[:, 'change_user'] = shift(df.user_id, -1, cval=0) - df['user_id']
    df.loc[df['change_user'] != 0, 'plugged_change'] = 0
    df = df.drop(columns=['change_user'])
    
    min_time = df.record_time.min()
    df.loc[:,'total_secs'] = (df.record_time - min_time).dt.total_seconds()
    last_total_secs = df.iloc[-1,:].total_secs
    df.loc[:,'changed_secs'] = shift(df.total_secs, -1, cval=last_total_secs) - df.total_secs
    df.loc[df['changed_secs'] > 18 * 3600, 'plugged_change'] = 0
    df = df.drop(columns=['record_time', 'total_secs', 'changed_secs'])
    
    charge_counts = df.groupby(['user_id', 'plugged_change']).size().reset_index(name='charge_count')
    charge_counts = charge_counts.drop(charge_counts[charge_counts.plugged_change != 1].index)
    charge_counts = charge_counts.drop(columns=['plugged_change'])
    
    return charge_counts

In [8]:
def num_charges_per_user_four_interval(df):
    df = df.set_index('record_time', inplace=False)
    
    df_1 = df.between_time('00:00:00', '05:59:59')
    df_2 = df.between_time('06:00:00', '11:59:59')
    df_3 = df.between_time('12:00:00', '17:59:59')
    df_4 = df.between_time('18:00:00', '23:59:59')
    
    df_1.reset_index(inplace=True)
    df_2.reset_index(inplace=True)
    df_3.reset_index(inplace=True)
    df_4.reset_index(inplace=True)
    
    charge_counts_1 = num_charges_per_user(df_1)
    charge_counts_2 = num_charges_per_user(df_2)
    charge_counts_3 = num_charges_per_user(df_3)
    charge_counts_4 = num_charges_per_user(df_4)
    
    charge_counts_1.columns = ['user_id', 'charge_count_1']
    charge_counts_2.columns = ['user_id', 'charge_count_2']
    charge_counts_3.columns = ['user_id', 'charge_count_3']
    charge_counts_4.columns = ['user_id', 'charge_count_4']
    
    charge_counts = pd.merge(charge_counts_1, charge_counts_2, on='user_id', how='outer')
    charge_counts = pd.merge(charge_counts, charge_counts_3, on='user_id', how='outer')
    charge_counts = pd.merge(charge_counts, charge_counts_4, on='user_id', how='outer')
    
    return charge_counts

In [9]:
# should deleter later
# excluded "start charge" events where the phone was already at 100%
def num_charges_per_user_v2(df):
    df.loc[:, 'inside_100_level'] = (df.level == 100) & (shift(df.level, -1, cval=0) == 100) & (shift(df.level, -2, cval=0) == 100) & (shift(df.level, -3, cval=0) == 100)
    df = df[df['inside_100_level'] == False]
    
    df.loc[df.plugged != 0, 'plugged'] = 1 
    df.sort_values(['user_id', 'record_time'])
    df = df.drop(columns=['record_time', 'level', 'accu','lat', 'lon'])
    
    df.loc[:, 'plugged_change'] = shift(df.plugged, -1, cval=0) - df['plugged']
    charge_counts = df.groupby(['user_id', 'plugged_change']).size().reset_index(name='charge_count')
    charge_counts = charge_counts.drop(charge_counts[charge_counts.plugged_change != 1].index)
    charge_counts = charge_counts.drop(columns=['plugged_change'])
    
    return charge_counts

In [10]:
def num_cells_charge_per_user(df):
    cells_df = df.groupby(['user_id', 'grid_x', 'grid_y']).sum().reset_index()
    cells_df = cells_df[cells_df['plugged'] > 0]
    
    return cells_df.groupby(['user_id']).size().reset_index(name='charge_cell_counts')

In [11]:
# First we keep 100 battery level while charging records as a separate bin
# Then we partition 0-100 into (n - 1) parts equally
# parameter df: dataframe
# parameter n: total number of bins
# return a dataframe with one row for each user and columns including: user_id and one column for one bin frequency
def battery_level_bins_percentage(df, n = 5):
    df.loc[:, 'which_bin'] = np.floor(df['level'] / (100 / (n - 1)))
    # Those 100 battery level but not charging should be in the penultimate bin
    df.loc[(df['level'] == 100) & (df['plugged'] == 0), 'which_bin'] = n - 2
    df['which_bin'] = df['which_bin'].astype('int')
    bins_df = df.groupby(by=['user_id', 'which_bin']).size().reset_index(name='count')
    
    bins_df.set_index('user_id', inplace=True)
    bins_df = bins_df.pivot(index=bins_df.index, columns='which_bin')['count']
    del bins_df.columns.name
    
    bins_df.loc[:, 'sum'] = bins_df[0]
    for i in range(1, n):
        bins_df['sum'] = bins_df['sum'] + bins_df[i]
        
    columns_name = [] 
    for i in range(0, n):
        columns_name.append('battery_level_bin_' + str(i) + '_perc')
        bins_df[i] = bins_df[i] / bins_df['sum']
        
    bins_df = bins_df.drop(columns=['sum'])
    bins_df.columns = columns_name
    bins_df.reset_index(inplace=True)
    
    return bins_df

In [12]:
home_charge_percentage = home_charge_percentage(battery_gps_df)

In [13]:
home_charge_percentage.head()

,user_id,home_charge_perc
0,514,0.681629
1,534,0.644766
2,551,0.899927
3,555,0.456202
4,559,0.970503


In [14]:
plug_counts = plug_in_time_percentage(battery_gps_df)

plugged       0       1      2
user_id                       
514      3877.0  2151.0   10.0
534      3678.0   923.0  873.0
551      3654.0  2708.0   40.0
555      4397.0  1701.0    NaN
559      3517.0  2068.0    NaN


In [15]:
plug_counts.head()

,user_id,uncharge_perc,charging_mode1_perc,charging_mode2_perc
0,514,0.642100,0.356244,0.001656
1,534,0.671904,0.168615,0.159481
2,551,0.570759,0.422993,0.006248
3,555,0.721056,0.278944,0.000000
4,559,0.629722,0.370278,0.000000


In [16]:
charge_counts = num_charges_per_user(battery_gps_df)

In [17]:
charge_counts.head()

,user_id,charge_count
2,514,83
5,534,100
8,551,56
11,555,57
14,559,35


In [18]:
# should delete later
charge_counts_2 = num_charges_per_user_v2(battery_gps_df)

/Users/wbb/anaconda3/envs/cmpt826/lib/python3.7/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [19]:
# should delete later
charge_counts_2.head()

,user_id,charge_count
2,514,83
5,534,100
8,551,56
11,555,57
14,559,36


In [20]:
charge_counts_four_interval = num_charges_per_user_four_interval(battery_gps_df)

In [21]:
# sum of them is larger the result above
charge_counts_four_interval.head()

,user_id,charge_count_1,charge_count_2,charge_count_3,charge_count_4
0,514,3.0,15,26,34.0
1,534,23.0,15,21,37.0
2,551,2.0,1,13,39.0
3,555,2.0,26,19,7.0
4,559,4.0,2,2,26.0


In [22]:
mean_length_charge = mean_length_charge_per_user(battery_gps_df)

In [23]:
mean_length_charge.head()

,user_id,charge_length
0,514,27.048193
1,534,18.772277
2,551,49.175439
3,555,30.842105
4,559,58.444444


In [24]:
charge_cell_counts = num_cells_charge_per_user(battery_gps_df)

In [25]:
charge_cell_counts.head()

,user_id,charge_cell_counts
0,514,33
1,534,13
2,551,17
3,555,17
4,559,2


In [26]:
battery_level_bins_perc = battery_level_bins_percentage(battery_gps_df, 3)

In [27]:
battery_level_bins_perc.head()

,user_id,battery_level_bin_0_perc,battery_level_bin_1_perc,battery_level_bin_2_perc
0,514,0.380755,0.411891,0.207353
1,534,0.484289,0.470954,0.044757
2,551,0.283037,0.422212,0.294752
3,555,0.233027,0.640538,0.126435
4,559,0.243688,0.462310,0.294002


In [50]:
df_for_classify = pd.merge(home_charge_percentage, plug_counts, on='user_id', how='outer')

In [51]:
df_for_classify = pd.merge(df_for_classify, charge_counts, on='user_id', how='outer')

In [52]:
df_for_classify = pd.merge(df_for_classify, mean_length_charge, on='user_id', how='outer')

In [53]:
df_for_classify = pd.merge(df_for_classify, charge_cell_counts, on='user_id', how='outer')

In [54]:
df_for_classify = pd.merge(df_for_classify, battery_level_bins_perc, on='user_id', how='outer')

In [55]:
#df_for_classify = pd.merge(df_for_classify, charge_counts_four_interval, on='user_id', how='outer')

In [56]:
#df_for_classify = df_for_classify.drop(columns=['charge_count'])

In [57]:
df_for_classify.head()

,user_id,home_charge_perc,uncharge_perc,charging_mode1_perc,charging_mode2_perc,charge_count,charge_length,charge_cell_counts,battery_level_bin_0_perc,battery_level_bin_1_perc,battery_level_bin_2_perc
0,514,0.681629,0.642100,0.356244,0.001656,83,27.048193,33,0.380755,0.411891,0.207353
1,534,0.644766,0.671904,0.168615,0.159481,100,18.772277,13,0.484289,0.470954,0.044757
2,551,0.899927,0.570759,0.422993,0.006248,56,49.175439,17,0.283037,0.422212,0.294752
3,555,0.456202,0.721056,0.278944,0.000000,57,30.842105,17,0.233027,0.640538,0.126435
4,559,0.970503,0.629722,0.370278,0.000000,35,58.444444,2,0.243688,0.462310,0.294002


In [58]:
df_for_classify.to_csv('data_for_classify.csv', index=False)

In [153]:
charge_counts.charge_count - charge_counts_2.charge_count

2       0
5       0
8       0
11      0
14     -1
17      0
20      1
23      3
26     -1
29     -1
32     -1
35     -1
38      0
41      0
44     10
47      5
50     -1
53      0
56      0
59      0
62      0
65      0
68      0
71     -2
74      0
77      0
80     -1
83      0
86      0
89      0
92      0
95      3
98      0
101     0
104    -1
107     2
110     0
113     0
116    -1
119     0
122     5
Name: charge_count, dtype: int64